In [ ]:
import adb
%matplotlib notebook
import image
import sys
from image import Image, Template
from matplotlib import pyplot as plt
from time import sleep, time
import numpy as np
import ipywidgets as widgets
from ipywidgets import HBox
from IPython.display import display
from threading import Thread
MARIAH_IP = "192.168.0.16"
TODD_IP = "192.168.1.112"
HOME_IP = "192.168.50.76"

ip = widgets.RadioButtons(options = [("Todd", TODD_IP),
                                     ("Mariah", MARIAH_IP),
                                     ("Home", HOME_IP)],
                          description = "WiFi")

def connect_click(b):
    with output:
        adb.connect(ip.value)
        
def disconnect_click(b):
    with output:
        adb.disconnect(ip.value)
        
def disconnect_all_click(b):
    with output:
        adb.disconnect()
adb.devices()
output = widgets.Output()
connect = widgets.Button(description = "Connect")
disconnect = widgets.Button(description = "Disconnect")
connect.on_click(connect_click)
disconnect.on_click(disconnect_click)
display(ip, connect, disconnect, output)

In [ ]:
screen = Image(adb.take_screenshot())


def double_tap(x,y,pause = 0):
    adb.tap(x,y).communicate()
    sleep(pause/1000.)
    adb.tap(x,y).communicate()
    
def x_tap(x,y,times,pause = 0):
    for i in range(times):
#         adb.tap(x,y)
        adb.swipe(x,y,x,y,dur = 65)
#         adb.tap(x,y).communicate()
        sleep(pause/1000.)
    

adb.devices()

def rapid_tap(x,y,pause = 50):
    while True:
        adb.tap(x,y)
        sleep(pause/1000.)

def run(locations,double_tap_pause = 200,rest=1000, iterations = 500, taps = 2):
    """locations should be an iterable of (x,y) tuples"""
    global kill
    for i in range(iterations):
        if kill:
            break
        print "\rOn cycle {:3d}/{:3d}".format(i + 1, iterations),
        sys.stdout.flush()
        for x,y in locations:
            if kill:
                break
            x_tap(x,y,taps,double_tap_pause)
            sleep(rest/1000.)
    else:
        print

    print "stopping"
            
def clear_coords(button):
    global coords
    coords = []
    print "cleared coords"
    
def onclick(event):
    coords.append([int(event.xdata),int(event.ydata)])
    with output:
        print "added {},{} to coordinates".format(*(coords[-1]))
        
def go_click(b):
    global coords
    global kill
    print "starting"
    kill = False
    TAP_DELAY = 100 # executes second tap as soon as process ends
    Thread(
        target = run,
        kwargs = {
            "taps": taps.value,
            "locations" : coords,
            "double_tap_pause" : TAP_DELAY,
            "rest" : loop_duration.value * 1000 / len(coords),
            "iterations" : times_to_harvest.value / len(coords)
        }
    ).start()
    
def stop_click(b):
    global kill
    kill = True
    print "\nkilling"
    
def refresh_click(button):
    global im
    im.set_data(adb.take_screenshot()[:,:,(2,1,0)])
    print "screen refreshed"
        
coords = []
TIMES_TO_HARVEST = 250
LOOP_DURATION = 30000
TAPS = 4
TAP_DELAY = 0 # executes second tap as soon as process ends

output = widgets.Output()
refresh = widgets.Button(description = "Refresh image")
clear = widgets.Button(description = "Clear")
go = widgets.Button(description = "go")
stop = widgets.Button(description = "stop")
taps = widgets.IntSlider(description = "times to tap", value=2,min=1,max=6)
loop_duration = widgets.FloatSlider(description = "cycle duration", value=10,min=0,max=10, step = 0.01)
times_to_harvest = widgets.IntSlider(description = "times to repeat", value=250,min=50,max=100000)
refresh.on_click(refresh_click)
clear.on_click(clear_coords)
go.on_click(go_click)
stop.on_click(stop_click)

            
f,a = plt.subplots()
f.set_figheight(4)
f.set_figwidth(9)
im = a.imshow(screen.im[:,:,(2,1,0)], )
display(refresh, clear, taps, loop_duration, times_to_harvest,go, stop, output)
f.canvas.mpl_connect('button_press_event', onclick)
f.show()

# rapid tapping

In [ ]:
screen = Image(adb.take_screenshot())

location = 0,0
    

def run(pause = 50):
    """locations should be an iterable of (x,y) tuples"""
    global kill
    global location
    count = 0
    while not kill:
        count += 1
        adb.tap(*location)
        sleep(pause/1000.)
        print "\rOn iteration {:10d}".format(count),
        sys.stdout.flush()

    print "\nstopping"
    
def onclick(event):
    global location
    location = (int(event.xdata),int(event.ydata))
        
def go_click(b):
    global coords
    global kill
    print "starting"
    kill = False
    TAP_DELAY = 0 # executes second tap as soon as process ends
    Thread(
        target = run,
        kwargs = {
            "pause": pause_text.value
        }
    ).start()
    
def stop_click(b):
    global kill
    kill = True
    print "\nkilling"
    
def refresh_click(button):
    global im
    im.set_data(adb.take_screenshot()[:,:,(2,1,0)])
    print "screen refreshed"
        
PAUSE = 50

refresh = widgets.Button(description = "Refresh image")
go = widgets.Button(description = "go")
stop = widgets.Button(description = "stop")
pause_slider = widgets.IntSlider(description = "period in ms", value = 100, min = 10, max = 1000)
pause_text = widgets.IntText(descriptsion = "period in ms", value = 100)
widgets.link((pause_slider, "value"), (pause_text, "value"))
refresh.on_click(refresh_click)
go.on_click(go_click)
stop.on_click(stop_click)

parameters = HBox((pause_slider, pause_text))
buttons = HBox((refresh, go, stop))

            
f,a = plt.subplots()
f.set_figheight(4)
f.set_figwidth(9)
im = a.imshow(screen.im[:,:,(2,1,0)], )
display(parameters, buttons)
f.canvas.mpl_connect('button_press_event', onclick)
f.show()